# WHATSAPP PARSER
This notebook parses the data collected in WhatsApp public groups, converting from free text format to structured data in a CSV table.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
import unicodedata, re, itertools, sys, os
import datetime
import codecs

In [2]:
#selects the database to be parsed
base = 2018

## Mapping dictionaries of phone numbers codes

In [3]:
# maps the DDD (regional code) and DDI (international code) to brazilian state or country, respectively

ddd2UF = {
    "11": "SP",
    "12": "SP",
    "13": "SP",
    "14": "SP",
    "15": "SP",
    "16": "SP",
    "17": "SP",
    "18": "SP",
    "19": "SP",
    "21": "RJ",
    "22": "RJ",
    "24": "RJ",
    "27": "ES",
    "28": "ES",
    "31": "MG",
    "32": "MG",
    "33": "MG",
    "34": "MG",
    "35": "MG",
    "37": "MG",
    "38": "MG",
    "41": "PR",
    "42": "PR",
    "43": "PR",
    "44": "PR",
    "45": "PR",
    "46": "PR",
    "47": "SC",
    "48": "SC",
    "49": "SC",
    "51": "RS",
    "53": "RS",
    "54": "RS",
    "55": "RS",
    "61": "DF",
    "62": "GO",
    "63": "TO",
    "64": "GO",
    "65": "MT",
    "66": "MT",
    "67": "MS",
    "68": "AC",
    "69": "RO",
    "71": "BA",
    "73": "BA",
    "74": "BA",
    "75": "BA",
    "77": "BA",
    "79": "SE",
    "81": "PE",
    "82": "AL",
    "83": "PB",
    "84": "RN",
    "85": "CE",
    "86": "PI",
    "87": "PE",
    "88": "CE",
    "89": "PI",
    "91": "PA",
    "92": "AM",
    "93": "PA",
    "94": "PA",
    "95": "RR",
    "96": "AP",
    "97": "AM",
    "98": "MA",
    "99": "MA"
  }

ddd2state = {
    "11": "São Paulo",
    "12": "São Paulo",
    "13": "São Paulo",
    "14": "São Paulo",
    "15": "São Paulo",
    "16": "São Paulo",
    "17": "São Paulo",
    "18": "São Paulo",
    "19": "São Paulo",
    "21": "Rio de Janeiro",
    "22": "Rio de Janeiro",
    "24": "Rio de Janeiro",
    "27": "Espírito Santo",
    "28": "Espírito Santo",
    "31": "Minas Gerais",
    "32": "Minas Gerais",
    "33": "Minas Gerais",
    "34": "Minas Gerais",
    "35": "Minas Gerais",
    "37": "Minas Gerais",
    "38": "Minas Gerais",
    "41": "Paraná",
    "42": "Paraná",
    "43": "Paraná",
    "44": "Paraná",
    "45": "Paraná",
    "46": "Paraná",
    "47": "Santa Catarina",
    "48": "Santa Catarina",
    "49": "Santa Catarina",
    "51": "Rio Grande do Sul",
    "53": "Rio Grande do Sul",
    "54": "Rio Grande do Sul",
    "55": "Rio Grande do Sul",
    "61": "Distrito Federal",
    "62": "Goiás",
    "63": "Tocantins",
    "64": "Goiás",
    "65": "Mato Grosso",
    "66": "Mato Grosso",
    "67": "Mato Grosso do Sul",
    "68": "Acre",
    "69": "Rondônia",
    "71": "Bahia",
    "73": "Bahia",
    "74": "Bahia",
    "75": "Bahia",
    "77": "Bahia",
    "79": "Sergipe",
    "81": "Pernambuco",
    "82": "Alagoas",
    "83": "Paraíba",
    "84": "Rio Grande do Norte",
    "85": "Ceará",
    "86": "Piauí",
    "87": "Pernambuco",
    "88": "Ceará",
    "89": "Piauí",
    "91": "Pará",
    "92": "Amazonas",
    "93": "Pará",
    "94": "Pará",
    "95": "Roraima",
    "96": "Amapá",
    "97": "Amazonas",
    "98": "Maranhão",
    "99": "Maranhão"
  }

ddi2Country = {
    1: 'ESTADOS UNIDOS',
    351: 'PORTUGAL',
    55: 'BRASIL',
    92: 'PAQUISTÃO',
    595: 'PARAGUAI',
    34: 'ESPANHA',
    39: 'ITÁLIA',
    44: 'ILHA DE MAN',
    49: 'ALEMANHA',
    597: 'SURINAME',
    58: 'VENEZUELA',
    33: 'FRANÇA',
    61: 'ILHAS COCOS (KEELING)',
    591:'BOLÍVIA',
    244: 'ANGOLA',
    54: 'ARGENTINA',
    212: 'MARROCOS',
    7: 'RÚSSIA',
    64:'NOVA ZELANDIA',
    91: 'ÍNDIA',
    27: 'ÁFRICA DO SUL',
    967: 'IÊMEN',
    20:'EGITO',
    90:'TURQUIA',
    258:'MOÇAMBIQUE',
    592:'GUIANA',
    229:'BENIM',
    57:'COLÔMBIA',
    32:'BÉLGICA',
    51:'PERU',
    47:'NORUEGA',
    966:'ARÁBIA SAUDITA',
    41: 'SUIÇA',
    233:'GANA',
    52:'MÉXICO',
    56:'CHILE',
    239:'SÃO TOMÉ E PRÍNCIPE',
    81: 'JAPÃO',
    98:'IRÃ',
    62:'INDONÉSIA',
    502:'GUATEMALA',
    31:'PAÍSES BAIXOS',
    94:'SRI LANKA',
    593:'EQUADOR',
    238:'CABO VERDE',
    598:'URUGUAI',
    46:'SUÉCIA'
}

country2ISO3 = {
    'ESTADOS UNIDOS':'USA',
    'PORTUGAL':'PRT',
    'BRASIL':'BRA',
    'PAQUISTÃO':'PAK',
    'PARAGUAI':'PRY',
    'ESPANHA':'ESP',
    'ITÁLIA':'ITA',
    'ILHA DE MAN':'IMN',
    'ALEMANHA':'DEU',
    'SURINAME':'SUR',
    'VENEZUELA':'VEN',
    'FRANÇA':'FRA',
    'ILHAS COCOS (KEELING)':'CCK',
    'BOLÍVIA':'BOL',
    'ANGOLA':'AGO',
    'ARGENTINA':'ARG',
    'MARROCOS':'MAR',
    'RÚSSIA':'RUS',
    'NOVA ZELANDIA':'NZL',
    'ÍNDIA':'IND',
    'ÁFRICA DO SUL':'ZAF',
    'IÊMEN':'YEM',
    'EGITO':'EGY',
    'TURQUIA':'TUR',
    'MOÇAMBIQUE':'MOZ',
    'GUIANA':'GUY',
    'BENIM':'BEN',
    'COLÔMBIA':'COL',
    'BÉLGICA':'BEL',
    'PERU':'PER',
    'NORUEGA':'NOR',
    'ARÁBIA SAUDITA':'SAU',
    'SUIÇA':'CHE',
    'GANA':'GHA',
    'MÉXICO':'MEX',
    'CHILE':'CHL',
    'SÃO TOMÉ E PRÍNCIPE':'STP',
    'JAPÃO':'JPN',
    'IRÃ':'IRN',
    'INDONÉSIA':'IDN',
    'GUATEMALA':'GTM',
    'PAÍSES BAIXOS':'NLD',
    'SRI LANKA':'LKA',
    'EQUADOR':'ECU',
    'CABO VERDE':'CPV',
    'URUGUAI':'URY',
    'SUÉCIA':'SWE'
}

print(len(ddi2Country))

47


## Pre-processing and parsing functions with regex

In [4]:
text_file = 'raw/2018/21.txt' #pre-processing example

#non-printable characters
nonprintable_chars = [u'\x00',u'\u200d',u'\u200e',u'\u202a',u'\u202c']
control_chars = ''.join(c for c in nonprintable_chars)
control_char_re = re.compile('[%s]' % re.escape(control_chars))

def remove_control_chars(s):
    return control_char_re.sub('', s)

def readAndProcessWppText(text_file):
    '''
    Reads and pre-processes a text file, cleaning characters and correcting dates 
    so they don't break the parsing done with regular expression
    
    Input: path to text file
    Output: pre-processed text
    
    '''
    with codecs.open(text_file,'r',encoding='utf-8') as f:
        text = f.read()
    #normalize text
    text = unicodedata.normalize("NFKC", text) #‘NFC’, ‘NFKC’, ‘NFD’ , ‘NFKD’.
    text = remove_control_chars(text)
    
    #preventing regex from break    
    #dates and one char
    text_dates = re.findall('\n\d+/\d+/\d+ \D',text)
    for td in text_dates:
        #print(td)
        text = re.sub('\n\d+/\d+/\d+ \D',td.replace(' ','***_space_***'), text, 1)
        
    #date and hour with 'h' or other char except ':'
    text_dates = re.findall('\n\d+/\d+/\d+ \d+[^:]\d+',text)
    for td in text_dates:
        #print(td)
        text = re.sub('\n\d+/\d+/\d+ \d+[^:]\d+',td.replace(' ','***_space_***'), text, 1)
        
    #date and hour with ':' and seconds
    text_dates = re.findall('\n\d+/\d+/\d+ \d+:\d+:\d+',text)
    for td in text_dates:
        #print(td)
        text = re.sub('\n\d+/\d+/\d+ \d+:\d+:\d+',td.replace(' ','***_space_***'), text, 1)
        
    #date and hour with ': and '\n'
    text_dates = re.findall('\n\d+/\d+/\d+ \d+:\d+\n',text)
    for td in text_dates:
        #print(td)
        text = re.sub('\n\d+/\d+/\d+ \d+:\d+\n',td.replace(' ','***_space_***'), text, 1)
    
    return text

text = readAndProcessWppText(text_file)
#text

In [5]:
#contatos adicionados
re.findall('\D+ adicionou \+\d+',text)
#re.findall('\+\d+',text)

[]

In [6]:
def splitDates(text):
    '''
    Use regex to split each message in dates and the rest (hour, phone and text)
    '''
    date = '\n\d+/\d+/\d+ '
    pattern = re.compile(date)
    dates = re.findall(pattern, text)
    meta_data = re.split(pattern, text)
    #remove initial messages when the user joins a group 
    #Ex: 07/04/20 07:24 - As mensagens deste grupo estão protegidas com a criptografia de ponta a ponta. Toque para mais informações.\n11/12/19 22:10 - \u200e+55 11 98085-5668 criou o grupo "Zuera24horas"\n07/04/20 07:23 - Você entrou usando o link de convite deste grupo
    #meta_data = meta_data[3:]
    dates = dates[2:]
    dates = [d.replace('\n','').strip() for d in dates]
    return dates , meta_data
dates , meta_data = splitDates(text)
print(len(dates))
print(len(meta_data))
#meta_data[0:10]

4875
4878


In [7]:
def filterMessages(dates, meta_data):
    '''
    Removes noisy messages that would break the regex: joined a group, leaved a group, changed group image, etc
    '''
    meta_data_filter = []
    dates_filter = []
    contacts_names = []
    dates_contatcs = []
    
    for m,d in zip(meta_data, dates):
        #entrou ou saiu
        noise = re.search('\d+ saiu', m)
        noise = noise or re.search('\d+ entrou usando o link de convite deste grupo', m)
        noise = noise or re.search('\d+ foi adicionado',m)
        noise = noise or re.search('\d+ removeu ', m)
        noise = noise or re.search('\d+ adicionou ', m)         
        noise = noise or re.search('\d+ - Você entrou usando o link de convite deste grupo',m)
        noise = noise or re.search('\d+ - Você saiu', m)
        #descrição do grupo
        noise = noise or re.search('\d+ mudou a descrição do grupo', m)
        noise = noise or re.search('\d+ alterou a descrição do grupo', m)        
        #mudou de número
        noise = noise or re.search('\d+ alterado para \+\d+', m)        
        #mudou imagem do grupo
        noise = noise or re.search('\d+ mudou a imagem deste grupo', m)
        noise = noise or re.search('\d+ alterou a imagem deste grupo', m)
        noise = noise or re.search('\d+ apagou a imagem deste grupo', m)
        #mudou telefone
        noise = noise or re.search('\d+ mudou para ', m)
        #mudou o nome do grupo
        noise = noise or re.search('\d+ mudou o nome de ', m)
        noise = noise or re.search('\d+ alterou o nome de ', m)
        #configurações do grupo
        noise = noise or re.search('\d+ mudou as configurações desse grupo para permitir que',m)
        noise = noise or re.search('\d+ alterou as configurações deste grupo para permitir que ',m)
        noise = noise or re.search('\d+ criou o grupo ', m)
        noise = noise or re.search('\d+ - As mensagens deste grupo estão protegidas com a criptografia de ponta a ponta. Toque para mais informações.',m)
        noise = noise or re.search('\d+ - As mensagens enviadas a este grupo estão agora protegidas com a criptografia de ponta-a-ponta. Toque para obter mais informações.',m) 
        noise = noise or re.search('\d+ apagou a descrição do grupo', m)
        noise = noise or re.search('\d+ - Você agora é um admin do grupo', m)
        
        #some groups can have users added as contacts. Here we save these messages for posterior processing
        is_contact = re.search('\d+:\d+ - [^+]\D+',m)         
        
        #get messages that aren't noise and aren't from contacts
        if not noise and not is_contact:
            meta_data_filter.append(m)
            dates_filter.append(d)
            
        #get messages from contacts
        if is_contact and not noise:
            contacts_names.append(m)
            dates_contatcs.append(d)            

    return meta_data_filter, dates_filter, list(zip(dates_contatcs,contacts_names))

meta_data_filter, dates_filter, contacts_names = filterMessages(dates, meta_data)
print(len(dates_filter))
print(len(meta_data_filter))
contacts_names

4685
4685


[]

In [8]:
def parseHours(meta_data_filter):
    '''
    Parses messages to get ddd, ddi, user_id, uf_br, country, country_iso
    '''
    
    regex = '\d+:\d+ - '
    pattern = re.compile(regex)
    hours = [re.findall(pattern,m)[0] for m in meta_data_filter]
    regex = '\d+:\d+'
    pattern = re.compile(regex)
    hours = [re.findall(pattern,m)[0] for m in hours]
    return hours
hours = parseHours(meta_data_filter)

In [9]:
def parsePhoneNumber(meta_data_filter):
    '''
    Parses messages to get phone number, ddd
    '''
    
    #DDD
    regex = ' - \+\d+ '
    pattern = re.compile(regex)
    phone_full = [re.split(pattern,m)[1].split(':')[0] for m in meta_data_filter]
    ddd = [m.split()[0] for m in phone_full]

    #DDI
    regex = ' - \+\d+'
    pattern = re.compile(regex)
    ddi = [re.findall(pattern,m)[0] for m in meta_data_filter]
    regex = '\+\d+'
    pattern = re.compile(regex)
    ddi = [re.findall(pattern,m)[0] for m in ddi]
    
    #ID
    user_id = [hash(phone) for phone in phone_full]
    
    #ESTADO
    uf_br = [ddd2state.get(d,'Estrangeiro') for d in ddd]
    
    #PAIS
    int_ddi = [int(s.replace('+','')) for s in ddi]
    country = [ddi2Country.get(d,'UNKNOWN') for d in int_ddi]
    country_iso = [country2ISO3.get(d,'UNKNOWN') for d in country]
    
    return ddd, ddi, user_id, uf_br, country, country_iso

ddd, ddi, user_id, uf_br, country, country_iso = parsePhoneNumber(meta_data_filter)

In [10]:
def haveURL(text):
    '''
    Tests if a string has an url
    Input: a string
    Output: 1 or 0
    '''
    indicadores = ['http','www','.com']
    for i in indicadores:
        if i in text:
            return 1
    return 0

def parseMessages(meta_data_filter):
    '''
    Get text content of a message
    '''
    regex = ' - \+\d+ '
    pattern = re.compile(regex)
    msg_phone = [re.split(pattern,m)[1] for m in meta_data_filter]
    msg = []
    midia = []
    has_url = []
    for m in msg_phone:
        #text
        parts = m.split(':')
        if len(parts) > 2:
            m = ':'.join(parts[1:])
        else:
            try:
                m = parts[1]
            except:
                print(m)
                
        m = m.strip()
        #corrects the edition made in pre-processing stage
        m = m.replace('***_space_***',' ')
        msg.append(m)
        
        #midia
        if '<Arquivo de mídia oculto>' in m:
            midia.append(1)
        else:
            midia.append(0)
        
        #url
        has_url.append(haveURL(m))        
        
        
    return msg, midia, has_url

msg, midia, has_url = parseMessages(meta_data_filter)

# Parsing function for all date in a file

In [11]:
def parse_wpp(text_file, gr):
    '''
    Do the parsing from a group (an txt file)
    
    Input: path to text file and alias for the group
    Output: pandas DataFrame with the columns: 'id', 'date', 'hour', 'ddi', 'ddd', 'state', 'country', 
                                                'country_iso3', 'midia', 'url', 'group', 'text'   
    
    '''    
    #get pre-processed text
    text = readAndProcessWppText(text_file)    
        
    #split date and meta-data
    dates , meta_data = splitDates(text)
    
    #filter noise
    meta_data_filter, dates_filter, contacts = filterMessages(dates, meta_data)
    
    #if there is no data in file
    if (len(meta_data_filter) < 1):
        df = pd.DataFrame(columns=['id', 'date', 'hour', 'ddi', 'ddd', 'state', 'country', 
                                   'country_iso3', 'midia', 'url', 'group', 'text'])
        return df, contacts    
        
    #get time
    hours = parseHours(meta_data_filter)    
    #parse phone
    ddd, ddi, users_id, uf_br, country, country_iso = parsePhoneNumber(meta_data_filter)
    #text
    msg, midia, has_url = parseMessages(meta_data_filter)
    #group
    grupos = [gr]*len(msg)
    
    #cria um dataframe        
    df = pd.DataFrame(list(zip(users_id, dates, hours, ddi, ddd, uf_br, country, 
                               country_iso, midia, has_url, grupos, msg)),
                      columns=['id', 'date', 'hour', 'ddi', 'ddd', 'state', 'country',
                               'country_iso3', 'midia', 'url', 'group', 'text'])
    return df, contacts

## Iterates over all files in a directory to build a DataFrame

In [12]:
path = 'raw/' + str(base) + '/'
    
df_n = pd.DataFrame(columns=['id', 'date', 'hour', 'ddi', 'ddd', 'state', 'country',
                             'country_iso3', 'midia', 'url', 'group', 'text'])
#saves contacts data
contacts = []
group_contacts = []
#aliases for groups
gr = 1

#iterates over files
for filename in os.listdir(path):
    print(filename,end = ' ')
    file_path = path + filename
    group = str(base) + '_' + str(gr)
    chat, cont = parse_wpp(file_path, group)
    gr += 1
    
    #if there is any useful data, append to dataframe
    if len(chat) > 0:
        df_n = df_n.append(chat, ignore_index=True)
        contacts += cont
        group_contacts += [group]*len(cont)

06.txt 21.txt 56.txt 11.txt 26.txt 43.txt 55.txt 41.txt 04.txt 20.txt 52.txt 02.txt 22.txt 35.txt 05.txt 54.txt 40.txt 07.txt 18.txt 49.txt 42.txt 17.txt 10.txt 30.txt 33.txt 01.txt 31.txt 03.txt 23.txt 44.txt 46.txt 09.txt 57.txt 08.txt 34.txt 39.txt 51.txt 53.txt 25.txt 19.txt 28.txt 24.txt 13.txt 15.txt 29.txt 32.txt 47.txt 48.txt 12.txt 14.txt 59.txt 45.txt 36.txt 58.txt 37.txt 16.txt 38.txt 50.txt 27.txt 

In [13]:
print(len(df_n))
df_n.head(5)  

277339


,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text
0,3631133147603888180,01/08/18,19:24,+55,17,São Paulo,BRASIL,BRA,0,0,2018_1,Aguardando esta mensagem
1,3631133147603888180,01/08/18,13:13,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>
2,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,0,0,2018_1,O Bolsonaro tem que estar preparado pra respon...
3,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>
4,-4391661641377612003,01/08/18,13:28,+55,13,São Paulo,BRASIL,BRA,0,0,2018_1,Boaaa


In [14]:
df_n.describe()

,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text
count,277339,277339,277339,277339,277339,277339,277339,277339,277339,277339,277339,277339
unique,5363,72,1440,19,125,28,19,19,2,2,59,110723
top,-3818310068976662355,07/10/18,20:03,+55,63,São Paulo,BRASIL,BRA,0,0,2018_54,<Arquivo de mídia oculto>
freq,4398,17318,443,275018,40991,44585,275018,275018,155512,250215,25175,121827


# TRATANDO CONTATOS ADICIONADOS

In [15]:
print(len(group_contacts))
print(len(contacts))

10756
10756


In [16]:
#edits names of contacts ending with a number:Ghost BA 5606
def editGhost(m):
    ghost = re.findall('Ghost \D+ \d+',m)
    if ghost:
        new_ghost = re.sub(' \d+','',ghost[0])
        new_msg = re.sub('Ghost \D+ \d+', new_ghost, m)
        return new_msg
    else:
        return m

if len(group_contacts) > 0:

    datas_c = [d[0] for d in contacts]
    meta_c = [m[1] for m in contacts]
    meta_c = [editGhost(m) for m in meta_c]

    #remove noise
    meta_c_filter = []
    datas_c_filter = []
    grupo_c_filter = []

    for m,d,g in list(zip(meta_c,datas_c, group_contacts)):
        content = re.search('^(\d+:\d+ - \D+: )',m)
        if content:
            meta_c_filter.append(m)
            datas_c_filter.append(d)
            grupo_c_filter.append(g)

    print(len(meta_c_filter))
    print(len(datas_c_filter))
    print(len(grupo_c_filter))

    #extract hours
    hours = [re.findall('^(\d+:\d+ -)',m)[0].replace(' -','') for m in meta_c_filter]
    print(len(hours))

    #extract names and text content
    nome_msg = [re.split('^(\d+:\d+ -)',m)[2].strip() for m in meta_c_filter]
    print(len(nome_msg))

    #anonimizes names
    nomes = [re.split(': ',m)[0].strip() for m in nome_msg]
    users_id = [hash(m) for m in nomes]
    print(len(nomes))
    set_nomes = set(nomes)

    #get text content
    mensagens = []
    for m in nome_msg:
        parts = m.split(': ')[1:]
        m = ': '.join(parts)
        m = m.strip()
        mensagens.append(m)

    print(len(mensagens))
    midia = [1 if '<Arquivo de mídia oculto>' in m else 0 for m in mensagens]
    has_url = [haveURL(m) for m in mensagens]
    ddd = ['?']*len(mensagens)

    df_adicionados = pd.DataFrame(list(zip(users_id, datas_c_filter, hours, ddd, ddd, ddd, ddd, 
                               ddd, midia, has_url, group_contacts, mensagens)),
                      columns=['id', 'date', 'hour', 'ddi', 'ddd', 'state', 'country',
                             'country_iso3', 'midia', 'url', 'group', 'text'])
    

    df = pd.concat([df_n, df_adicionados])
df.tail(5)

10498
10498
10498
10498
10498
10498
10498


,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text
10493,-3459365403520904542,28/10/18,21:48,?,?,?,?,?,1,0,2018_59,<Arquivo de mídia oculto>
10494,-3459365403520904542,28/10/18,22:06,?,?,?,?,?,1,0,2018_59,<Arquivo de mídia oculto>
10495,-3459365403520904542,28/10/18,22:08,?,?,?,?,?,1,0,2018_59,<Arquivo de mídia oculto>
10496,-3459365403520904542,28/10/18,22:12,?,?,?,?,?,1,0,2018_59,<Arquivo de mídia oculto>
10497,-3459365403520904542,28/10/18,23:12,?,?,?,?,?,1,0,2018_59,<Arquivo de mídia oculto>


In [17]:
len(df)

287837

## TIMESTAMP

In [18]:
def custom_time(data,hora):
    string = data + " " + hora + ":00"
    return string

df['timestamp'] = df.apply(lambda x: custom_time(x['date'],x['hour']),axis=1)
df['timestamp'] = pd.to_datetime(df['timestamp'])

## TEXT PROCESSING

In [19]:
df['text'] = [str(msg) for msg in list(df['text'])] 
df['text'] = [msg.strip() for msg in list(df['text'])]
df = df[df['text'] != 'Mensagem incompatível']
df = df[df['text'] != 'Esta mensagem foi apagada']
df = df[df['text'] != 'Essa mensagem foi apagada']
df = df[df['text'] != 'Aguardando esta mensagem']
df = df[df['text'] != 'nan']
df = df[df['text'] != '']

In [20]:
print(len(df))
df.head()

282681


,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text,timestamp
1,3631133147603888180,01/08/18,13:13,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:13:00
2,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,0,0,2018_1,O Bolsonaro tem que estar preparado pra respon...,2018-01-08 13:24:00
3,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:24:00
4,-4391661641377612003,01/08/18,13:28,+55,13,São Paulo,BRASIL,BRA,0,0,2018_1,Boaaa,2018-01-08 13:28:00
5,-4391661641377612003,09/08/18,14:46,+55,13,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-09-08 14:46:00


## Size of text

In [21]:
msgs = df['text']
len_msgs = [len(msg.split()) for msg in msgs]
df['words'] = len_msgs

len_msgs = [len(msg) for msg in msgs]
df['characters'] = len_msgs
df.head()

,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text,timestamp,words,characters
1,3631133147603888180,01/08/18,13:13,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:13:00,4,25
2,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,0,0,2018_1,O Bolsonaro tem que estar preparado pra respon...,2018-01-08 13:24:00,9,58
3,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:24:00,4,25
4,-4391661641377612003,01/08/18,13:28,+55,13,São Paulo,BRASIL,BRA,0,0,2018_1,Boaaa,2018-01-08 13:28:00,1,5
5,-4391661641377612003,09/08/18,14:46,+55,13,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-09-08 14:46:00,4,25


## Repeated texts

In [22]:
# only counts texts with more than 5 words
groupedByMsg = df[(df['words'] > 5) & (df['midia']==0)].groupby(by=['text']).count()
groupedByMsg = groupedByMsg.sort_values('url', ascending=False)['url']
groupedByMsg = pd.DataFrame(groupedByMsg)
groupedByMsg = groupedByMsg.rename(columns={'url':'COUNT'})
groupedByMsg = groupedByMsg[groupedByMsg['COUNT'] > 1]
groupedByMsg[0:10]

,COUNT
text,
"_*TSE informa:*_ 7,2 milhões de votos anulados pelas urnas! A diferença de votos que levaria à vitória de Bolsonaro no primeiro turno foi de menos de 2 milhões .\nO TSE tem obrigação de esclarecer os motivos que levaram à anulação de mais de 7,2 milhões de votos que representam 6,2% do total. A anulação só pode acontecer em voto de papel, porque permite rasuras ou ambiguidade.\n\nSe você enviar para apenas 20 contatos em um minuto, o Brasil inteiro vai desmascarar este Bandido. NÃO quebre essa corrente. Os incautos precisam ser esclarecidos antes que seja tarde demais...",91
"Sem palavras, só divulgue para seus contatos e peçam para que todos façam o mesmo simultaneamente e daqui um minuto está mensagem estará nos quatros cantos da terra.",88
"Vamos fazer campanha inteligente e garantir a vitória!?\n- Não compartilhem nada ofensivo ao Nordeste!!! 2014 internet bombou de ofensas e a Dilma cresceu absurdamente la no segundo turno!\n- Compartilhe mensagens positivas dele e da mudança!\n- Guarde pra VC o medo, transmita confiança para cativar votos!\n- Respeite a opinião alheia!\n- Cheque a Notícia, Fake News joga contra!\nEleição tem dois turnos essa é a regra do jogo! AGORA É A HORA!",86
Enquete para presidência! *Quem é seu candidato?* Vote e veja quem esta ganhando as Eleições 2018 *Acesse* ⤵\nhttps://pesquisaeleitoral2018.online/eleitoral/2018/,84
"Vamos dar ""dislike"" nos vídeos dos artistas ""rouanet's"" ""EleNão""\nClica no link, vai aparecer o vídeo e vc clica na mãozinha 👎🏼. A diferença do 👎🏼 para o 👍🏼 é gigantesca.\n\nAnitta\nhttps://youtu.be/QmrnZobpyW8\n\nDaniella Mercury\nhttps://youtu.be/GlrKJBfiXcI\n\nLetícia Sabatella\nhttps://youtu.be/83Z0ZxqJvPA\n\nLetícia Colin\nhttps://youtu.be/gtj9Ylc3Chc\n\nMarília Mendonça \nhttps://youtu.be/jQGtJq3yZh0\n\nClipe ""EleNão""\nhttps://youtu.be/SBBVS84oYP8",73
"Você foi selecionado para participar da *Nova pesquisa eleitoral* da _DATAFOLHA_, *responda* no link a seguir: http://datafolha.pesquisabr.site\nSua opinião é muito importante para o Brasil.",66
Vota aí e repassa!!! Vamos ver se o ibope está certo?\n\n\nhttps://pt.surveymonkey.com/r/W85R38F,66
"Nota Oficial :\n\nPessoal aqui quem lhe falam é Eduardo Bolsonaro, candidato a deputado federal.\n\nPessoal este é um recadinho muito rápido, peço a todos que compartilhem com o máximo de pessoas ! Compartilhem em grupos ! Em contatos, nas mídias sociais.\n\nPessoal, amanhã dia 07/10 dia de eleição ! Não compareça a sua seção com camiseta de apoio ao Jair Bolsonaro. \n\nPessoal em todos estes anos de eleição, nunca foi admitido este tipo de manifestação, mesmo silenciosa, é considerado boca de urna, é considerado CRIME !\n\nEstá muito estranho está historinha de poder ir votar com a camiseta do candidato. Isto está nós cheirando golpe, para anular o voto de vocês ! \n\nEntão amanhã não compareçam com a camiseta em apoio ao candidato Jair BOLSONARO, vamos ir de amarelo, camiseta do Brasil, mas por favor, vamos desconfiar de tudo ! Isso está nos gerando golpe no TSE contra a vitória em primeiro turno de Bolsonaro. \n\nEles podem não anular 100% dos votos, mas se eles anularem 15% já perdemos em primeiro turno. \n\nEntão desconfiem de tudo. \n\nAgradeço a todos que compartilharem essa mensagem.\n\nEduardo Bolsonaro,\nBrasil acima de tudo, Deus Acima de Todos.\n\n\n#17",63
"Olha só que ideia interessante...\n\nSe formos 75 milhões de filiados ao psl, obrigatoriamente terão que haver no mínimo 75 milhões de votos para Bolsonaro nas urnas...\n\nAí não tem como fraudar...\n👇👇👇👇👇👇👇\n\n\nGalera estamos em uma campanha massiva de cadastro de filiação no site do PSL, é de graça, pegue seu RG, CPF e Titulo de Eleitor - queremos passar dos 50 milhões de inscritos -, assim poderemos contrapor a fraude do voto contra o TSE, entre no link, faça seu cadastro e divulgue em toda rede social, convoque os demais. 👇😎🙏🇧🇷🇧🇷😎👇Vamos COMPARTILHAR em todas as redes sociais!\n\nhttps://www.pslnacional.org.br/\n\nPrecisamos alcançar 75 milhões d

In [23]:
fowarded = list(groupedByMsg.index) 
qnt = list(groupedByMsg['COUNT'])
shared = pd.DataFrame({'MSG':fowarded,'COUNT':qnt})
shared.head()

,MSG,COUNT
0,"_*TSE informa:*_ 7,2 milhões de votos anulados...",91
1,"Sem palavras, só divulgue para seus contatos e...",88
2,Vamos fazer campanha inteligente e garantir a ...,86
3,Enquete para presidência! *Quem é seu candidat...,84
4,"Vamos dar ""dislike"" nos vídeos dos artistas ""r...",73


## Create field of number of sharings if it is viral (shared more than once)

In [24]:
def putLabelIfInList(instance,l):
    if instance in l:
        return 1
    else:
        return 0    

fowarded = list(groupedByMsg.index)       
df['viral'] = df.apply(lambda x: putLabelIfInList(x['text'],fowarded),axis=1)

def labelSharings(instance,df):
    l = list(df['MSG'])
    try:
        loc = l.index(instance)
        qnt = df['COUNT'].iloc[loc]
    except:
        qnt = 1
    return qnt

df['sharings'] = df.apply(lambda x: labelSharings(x['text'],shared),axis=1)

df.head()

,id,date,hour,ddi,ddd,state,country,country_iso3,midia,url,group,text,timestamp,words,characters,viral,sharings
1,3631133147603888180,01/08/18,13:13,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:13:00,4,25,0,1
2,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,0,0,2018_1,O Bolsonaro tem que estar preparado pra respon...,2018-01-08 13:24:00,9,58,1,2
3,3631133147603888180,01/08/18,13:24,+55,17,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-01-08 13:24:00,4,25,0,1
4,-4391661641377612003,01/08/18,13:28,+55,13,São Paulo,BRASIL,BRA,0,0,2018_1,Boaaa,2018-01-08 13:28:00,1,5,0,1
5,-4391661641377612003,09/08/18,14:46,+55,13,São Paulo,BRASIL,BRA,1,0,2018_1,<Arquivo de mídia oculto>,2018-09-08 14:46:00,4,25,0,1


In [25]:
#reorganizing coluns
cols = df.columns.tolist()
cols = ['id',
        'date',
        'hour',
        'ddi',
        'country',
        'country_iso3',
        'ddd',
        'state',
        'group',
        'midia',
        'url',
        'characters',
        'words',        
        'viral',
        'sharings',
        'text']
df = df[cols]
#df = df.sort_values(by='ENCAMINHAMENTOS',ascending=False)
df.head()

,id,date,hour,ddi,country,country_iso3,ddd,state,group,midia,url,characters,words,viral,sharings,text
1,3631133147603888180,01/08/18,13:13,+55,BRASIL,BRA,17,São Paulo,2018_1,1,0,25,4,0,1,<Arquivo de mídia oculto>
2,3631133147603888180,01/08/18,13:24,+55,BRASIL,BRA,17,São Paulo,2018_1,0,0,58,9,1,2,O Bolsonaro tem que estar preparado pra respon...
3,3631133147603888180,01/08/18,13:24,+55,BRASIL,BRA,17,São Paulo,2018_1,1,0,25,4,0,1,<Arquivo de mídia oculto>
4,-4391661641377612003,01/08/18,13:28,+55,BRASIL,BRA,13,São Paulo,2018_1,0,0,5,1,0,1,Boaaa
5,-4391661641377612003,09/08/18,14:46,+55,BRASIL,BRA,13,São Paulo,2018_1,1,0,25,4,0,1,<Arquivo de mídia oculto>


In [26]:
len(df)

282681

In [27]:
#saves as CSV
filename = r'data/' + str(base) + '/wpp_'+ str(base) + '_stage1.csv'
print(filename)

data/2018/wpp_2018_stage1.csv


In [28]:
df.to_csv(filename, index=False)

# NEXT STAGE: labeling and anonymization